In [428]:
function fixBSP(BSPs, xbar)
    for s = 1:length(BSPs)
        BSP = BSPs[s]
        x = getindex(BSP, :xbar)
        for i = 1:length(xbar)
            setupperbound(x[i],xbar[i])
            setlowerbound(x[i],xbar[i])
        end
    end
end

function fixLSP(LSPs, origObjs, H, λ)
    for s = 1:length(LSPs)
        LSP = LSPs[s]
        origObj = origObjs[s]
        x = getindex(LSP, :x)
        @objective(LSP, Min, origObj + sum(x.*H[s]*λ[s]))
    end
    return λ
end

fixLSP (generic function with 2 methods)

In [429]:
function initialize(BSPs, LSPs, H)
    origObjs = []
    fixBSP(BSPs, [0,0])
    for s = 1:length(LSPs)
        push!(origObjs, LSPs[s].obj)
    end
    fixLSP(LSPs, origObjs, H, [0,0])
    return origObjs, [0,0]
end

function BSPsolve(BSPs)
    xhat = []
    zk = []
    μ = []
    for s = 1:length(BSPs)
        BSP = BSPs[s]
        solve(BSP)
        
        xbar = getvalue(getindex(BSP, :xbar))
        push!(xhat, xbar)
        
        z = getobjectivevalue(BSP)
        push!(zk, z)
        
        dual1 = getdual(getindex(BSP, :dual1))
        dual2 = getdual(getindex(BSP, :dual2))
        
        push!(μ, [dual1, dual2])
        
    end
    return μ, zk, xhat
end

function LSPsolve(LSPs)
    xtilde = []
    ytilde = []
    for s = 1:length(BSPs)
        LSP = LSPs[s]
        solve(LSP)
        
        x = getindex(LSP, :x)
        push!(xtilde, getvalue(x))
        
        y = getindex(LSP, :y)
        push!(ytilde, getvalue(y))
    end
    return xtilde, ytilde
end

function BSPtoLMP(LMP::JuMP.Model, zk, H, xhat)
    κ = getindex(LMP, :κ)
    λ = getindex(LMP, :λ)
    for s = 1:length(zk)
        @constraint(LMP, κ[s] <= zk[s]+sum(λ[s].*H[s].*xhat[s]))
    end
end

function BSPtoBMP(BSPs, BMP, μ)
    θ = getindex(BMP, :θ)
    x = getindex(BMP, :x)
    for s = 1:length(BSPs)
        BSP = BSPs[s]
        xbar = getvalue(getindex(BSP, :xbar))
        @constraint(BMP, θ[s] >= getobjectivevalue(BSP) - sum(μ[s].*xbar) + sum(μ[s].*x))

    end
end

function LSPtoLMP(LMP, τ, cT, dT, H, xtilde, ytilde)
    κ = getindex(LMP, :κ)
    λ = getindex(LMP, :λ)
    
    for s = 1:length(xtilde)
        @constraint(LMP, κ[s] <= sum(τ[s]*cT.*xtilde[s])+sum(τ[s]*dT[s].*ytilde[s])+λ[s]*sum(H[s].*xtilde[s]))
    end
end

function LSPtoBMP(LSPs, BMP, H, λ)
    θ = getindex(BMP, :θ)
    x = getindex(BMP, :x)
    for s = 1:length(LSPs)
        LSP = LSPs[s]
        @constraint(BMP, θ[s] >= getobjectivevalue(LSP)-sum(λ[s].*H[s].*x))
    end
        
end
    
function LMPtoLSP(LMP, LSPs, origObjs, H)
    solve(LMP)
    λ = getvalue(getindex(LMP, :λ))
    fixLSP(LSPs, origObjs, H, λ)
    return λ
end



function BMPtoBSP(BMP, BSPs)
    print(BMP)
    solve(BMP)
    x = getindex(BMP, :x)
    fixBSP(BSPs, getvalue(x))
end

BMPtoBSP (generic function with 1 method)

In [430]:
function crossDecomposition(BSPs, LSPs, τ, cT, dT, H, maxIterations)
    origObjs, λ = initialize(BSPs, LSPs, H)
    for i = 1:maxIterations
        μ, zk, xhat = BSPsolve(BSPs)
        BSPtoBMP(BSPs, BMP, μ)
        BSPtoLMP(LMP, zk, H, xhat)
        xtilde, ytilde = LSPsolve(LSPs)
        LSPtoLMP(LMP, τ, cT, dT, H, xtilde, ytilde)
        LSPtoBMP(LSPs, BMP, H, λ)
        BMPtoBSP(BMP, BSPs)
        print("h8")
        λ = LMPtoLSP(LMP, LSPs, origObjs, H)
        print("h9")
    end
    println(getvalue(getindex(BMP, :x)))
end

crossDecomposition (generic function with 1 method)

In [431]:
using JuMP
using Gurobi

LMP = Model(solver = GurobiSolver())
LSP1 = Model(solver = GurobiSolver())
LSP2 = Model(solver = GurobiSolver())
BMP = Model(solver = GurobiSolver())
BSP1 = Model(solver = GurobiSolver())
BSP2 = Model(solver = GurobiSolver())

τ = [0.5,0.5]
cT = [1, 2]
dT = [[3],[4]]
H = [[1, 1], [-1, -1]]

@variable(LSP1, x[1:2]>=0)
@constraint(LSP1, x[1:2].<=10)
@variable(LSP1, y>=0)
@constraint(LSP1, x[1]+x[2]+y >=6)
@objective(LSP1, Min, sum(τ.*cT.*x)+sum(dT[1].*y))

@variable(LSP2, x[1:2]>=0)
@constraint(LSP2, x[1:2].<=10)
@variable(LSP2, y>=0)
@constraint(LSP2, -3*x[1]+2*x[2]+y >=7)
@objective(LSP2, Min, sum(τ.*cT.*x)+sum(dT[2].*y))

@variable(BMP, θ[1:2]>=0)
@variable(BMP, x[1:2]>=0)
@constraint(BMP, x[1:2].<=10)
@objective(BMP, Min, sum(θ))

@variable(LMP, κ[1:2])
@variable(LMP, η)
@variable(LMP, λ[1:2])
@constraint(LMP, η <= sum(κ))
@objective(LMP, Max, η)

@variable(BSP1, x[1:2])
@variable(BSP1, xbar[1:2])
@variable(BSP1, y>=0)
@constraint(BSP1, x[1]+x[2]+y >=6)
@constraint(BSP1, dual1, x[1]-xbar[1] == 0)
@constraint(BSP1, dual2, x[2]-xbar[2]==0)
@objective(BSP1, Min, sum(τ.*cT.*x)+sum(dT[1].*y))

@variable(BSP2, x[1:2])
@variable(BSP2, xbar[1:2])
@variable(BSP2, w>=0)
@constraint(BSP2, -3*x[1]+2*x[2]+w >=7)
@constraint(BSP2, dual1, x[1]-xbar[1] == 0)
@constraint(BSP2, dual2, x[2]-xbar[2]==0)
@objective(BSP2, Min, sum(τ.*cT.*x)+sum(dT[2].*w))

LSPs = [LSP1, LSP2]
BSPs = [BSP1, BSP2]

2-element Array{JuMP.Model,1}:
 Minimization problem with:
 * 3 linear constraints
 * 5 variables
Solver is Gurobi
 Minimization problem with:
 * 3 linear constraints
 * 5 variables
Solver is Gurobi

In [432]:
crossDecomposition(BSPs, LSPs, τ, cT, dT, H, 2)

Academic license - for non-commercial use only
Optimize a model with 3 rows, 5 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.800000000e+01
Academic license - for non-commercial use only
Optimize a model with 3 rows, 5 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    

In [433]:
print(BMP)
print(LMP)

Min θ[1] + θ[2]
Subject to
 x[1] <= 10
 x[2] <= 10
 θ[1] + 2.5 x[1] + 2 x[2] >= 18
 θ[2] - 12.5 x[1] + 7 x[2] >= 28
 θ[1] >= 3
 θ[2] >= 3.5
 θ[1] - 0.5 x[1] - x[2] >= 0
 θ[2] - 0.5 x[1] - x[2] >= 0
 θ[1] + 2.5 x[1] + 2.5 x[2] >= 18
 θ[2] + 7 x[1] + 7 x[2] >= 28
 θ[i] >= 0 for all i in {1,2}
 x[i] >= 0 for all i in {1,2}
Max η
Subject to
 η - κ[1] - κ[2] <= 0
 κ[1] <= 18
 κ[2] <= 28
 κ[1] - 6 λ[1] <= 3
 κ[2] + 3.5 λ[2] <= 3.5
 κ[1] - 10 λ[1] <= 10
 κ[2] + 10 λ[2] <= 10
 κ[1] - 6 λ[1] <= 3
 κ[2] + 3.5 λ[2] <= 3.5
 κ[i] for all i in {1,2}
 λ[i] for all i in {1,2}
 η
